In [1]:
from QAOA_ansatz import create_QAOA_ansatz, create_hhl_circ
from amp_est import real_amp_est
from taylor_precomputation import construct_asin_x_inv_circuit, construct_exp_k_abs_cos_circuit
from problem_formulation import Node, Line, Grid, UCProblem

In [2]:
import numpy as np
from math import pi
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit.providers.aer import Aer
from qiskit.circuit import ParameterVector
from qiskit.circuit.library.data_preparation.state_preparation import StatePreparation

from qiskit.opflow import PauliTrotterEvolution
from qiskit.opflow import I, X, Y, Z

In [3]:
node1=Node([2,2], 5, 1, 1, "gen1")
node2=Node([1,1], 1, 2, 1, "gen2")
node3=Node([-1.5,-1], 0,0,0, "load1")
node4=Node([-1,0], 0,0,0, "load2")
line1=Line(node1,node3,1,1)
line2=Line(node2,node1,1,1)
line3=Line(node4,node2,1,1)
line4=Line(node4,node3,1,1)

problem_instance=UCProblem([line1,line2,line3, line4], [node1,node2,node3,node4], 2)

In [4]:
grid=problem_instance.grid_timesteps
grid.A

array([[ 3., -1.,  0., -1.],
       [-1.,  2., -1.,  0.],
       [ 0., -1.,  2., -1.],
       [-1.,  0., -1.,  2.]])

In [5]:
grid.A_eig_bounds

(0.18644324808654034, 5.0)

In [6]:
real_powers=np.array([node.real_power for node in grid.nodes])
print(real_powers[:,0])

[ 2.   1.  -1.  -1.5]


In [7]:
gen_nodes=QuantumRegister(2,"gen_nodes")
tot_nodes=QuantumRegister(2,"tot_nodes")
state_prep_anc=QuantumRegister(1,"state_prep_anc")
hhl_phase_reg=QuantumRegister(4,"hhl_phase_reg")
hhl_anc=QuantumRegister(1,"hhl_anc")

hhl_circ=[create_hhl_circ(real_powers=real_powers[:,0],
                         B=grid.A,
                         max_eigval=grid.A_eig_bounds[1],
                         C=grid.A_eig_bounds[0],
                         gen_nodes=gen_nodes,
                         tot_nodes=tot_nodes,
                         state_prep_anc=state_prep_anc,
                         hhl_phase_reg=hhl_phase_reg,
                         hhl_anc=hhl_anc)]

Check if HHL circuit already exists
'HHL_[ 2.   1.  -1.  -1.5]_[[ 3. -1.  0. -1.]\n [-1.  2. -1.  0.]\n [ 0. -1.  2. -1.]\n [-1.  0. -1.  2.]]_5.0_0.18644324808654034_4_3'
Constructing HHL Circuit
Initial B: [[ 3. -1.  0. -1.]
 [-1.  2. -1.  0.]
 [ 0. -1.  2. -1.]
 [-1.  0. -1.  2.]]
Final B: [[ 3.29472415 -1.09824138  0.         -1.09824138]
 [-1.09824138  2.19648277 -1.09824138  0.        ]
 [ 0.         -1.09824138  2.19648277 -1.09824138]
 [-1.09824138  0.         -1.09824138  2.19648277]]
Constructing asin(1/(mx)) circuit
Get asin(x^-1) expression
	Power expansion


KeyboardInterrupt: 

In [ ]:
# hhl_output_reg=ClassicalRegister(2, "hhl_output")
# hhl_circ[0].add_register(hhl_output_reg)
# # hhl_circ[0].measure(tot_nodes,hhl_output_reg)

In [ ]:
backend=Aer.get_backend('aer_simulator')
hhl_circ[0].save_statevector()
hhl_circ[0]=transpile(hhl_circ[0], backend)   
result = backend.run(hhl_circ[0]).result()
statevector = result.get_statevector(hhl_circ[0])

In [ ]:
# counts
# # {'00': 490, '11': 283, '10': 124, '01': 127}


In [ ]:
list(statevector)[:2**4]

C:\Users\Dell\AppData\Local\Temp/ipykernel_33084/3448962520.py:1: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  list(statevector)[:2**4]


[(1.1733252601426779e-15-7.093638722529953e-16j),
 (3.1238120490731777e-32+1.6032844265394164e-32j),
 (3.1238120490731777e-32+1.6032844265394164e-32j),
 (2.422002308634285e-49-2.2585817534952472e-48j),
 (2.351288456437307e-15-6.734635177308739e-16j),
 (-1.7372275174312124e-31+7.259382625465279e-32j),
 (-1.7372275174312124e-31+7.259382625465279e-32j),
 (1.1429597541803188e-47-7.298173540834337e-48j),
 (-0.24358325729500183+0.243583257295017j),
 (1.4915172818610517e-17-1.49151728186113e-17j),
 (1.4915172818610517e-17-1.49151728186113e-17j),
 (-9.132909325520507e-34+9.132909325521008e-34j),
 (-0.36537488594250805+0.3653748859425251j),
 (2.2372759227915947e-17-2.2372759227917066e-17j),
 (2.2372759227915947e-17-2.2372759227917066e-17j),
 (-1.3699363988280913e-33+1.3699363988281571e-33j)]

In [ ]:
list(statevector)[2**4:2**5]

C:\Users\Dell\AppData\Local\Temp/ipykernel_33084/247910180.py:1: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  list(statevector)[2**4:2**5]


[(0.4871665145900065-0.48716651459000543j),
 (-2.983034563722107e-17+2.983034563722117e-17j),
 (-2.983034563722107e-17+2.983034563722117e-17j),
 (1.8265818651041047e-33-1.826581865104084e-33j),
 (0.2435832572950146-0.24358325729501298j),
 (-1.4915172818611556e-17+1.4915172818611112e-17j),
 (-1.4915172818611556e-17+1.4915172818611112e-17j),
 (9.132909325520898e-34-9.132909325520713e-34j),
 (3.1223219952359198e-15-3.227717756860098e-15j),
 (-1.3948287500486283e-31+1.6232429428408943e-31j),
 (-1.3948287500486283e-31+1.6232429428408943e-31j),
 (1.1277392198338383e-47-1.128949637742376e-47j),
 (1.8131618488201636e-15-2.0853799341794943e-15j),
 (-1.1781751920259435e-31+1.2631962650550467e-31j),
 (-1.1781751920259435e-31+1.2631962650550467e-31j),
 (6.874682524022374e-48-7.441555789351929e-48j)]

In [ ]:
[statevector[0],
statevector[4],
statevector[8],
statevector[12]]

# array([ 0.5  ,  0.375, -0.75 , -0.875])
# [(1.1733252601426779e-15-7.093638722529953e-16j),
#  (2.351288456437307e-15-6.734635177308739e-16j),
#  (-0.24358325729500183+0.243583257295017j),
#  (-0.36537488594250805+0.3653748859425251j)]

[(1.1733252601426779e-15-7.093638722529953e-16j),
 (2.351288456437307e-15-6.734635177308739e-16j),
 (-0.24358325729500183+0.243583257295017j),
 (-0.36537488594250805+0.3653748859425251j)]

In [ ]:
A_inv=np.linalg.inv(grid.A)
A_inv @ real_powers[:,0]

array([ 0.5  ,  0.375, -0.75 , -0.875])